In [18]:
# importations
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn import metrics

import mlflow
from pyngrok import ngrok

from sklearn.cluster import KMeans

In [8]:
# chargement des données
data = pd.read_csv('Outputs/data.csv')

In [9]:
data.isnull().sum()

lib_etendu       0
bilan            0
montant_signe    0
signe            0
num_oper         0
diff_ope_val     0
dtype: int64

# On va maintenant creer un pipeline pour le preprocessing des données et le model

In [10]:
# On va recuperer les colonnes numeriques et categoricielles des donnees

# On recupere les colonnes numeriques
numerical_features = data.select_dtypes(exclude=['object']).columns

# On recupere les colonnes categoricielles
categorical_features = data.select_dtypes(include=['object']).columns

In [11]:
# On va creer une foncion global qui nous permettra de creer un pipeline avec des caracteristiques differentes

def create_pipeline(model = None, transform_for_num = None):

    numerical_pipeline = make_pipeline(
        SimpleImputer(strategy='most_frequent'),
        transform_for_num
    )
    # On ne fera plus de transformations sur les variables categoricielles

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_pipeline, numerical_features)
        ]
    )
    pipeline = Pipeline(
        steps=[
            ('preprocessor', preprocessor),
            ('model', model)
        ]
    )
    return pipeline

# On va maintenant creer des experiences avec MLflow afin de determiner le meilleur model

On va etudier differents models :
- Clustering-Based Local Outlier Factor
- Isolation Forest
- One-Class SVM
- K-Means
- DBSCAN
- Autoencoder

Nous utiliserons un mode d'apprentissage non supervisee pour detecter les anomalies. Donc pour les metrics d'evaluation, nous allons utiliser :
- silhouette_score : mesure la distance entre chaque point de données et les points de données de son cluster voisin le plus proche par rapport à la distance moyenne de tous les points de données dans le cluster. Un score élevé indique une bonne séparation des clusters.
- inertia : la quantité de variance des données qui est expliquée par les clusters. Plus l'inertie est faible, plus les clusters sont compacts et homogènes
- calinski_harabasz_score :  mesure la séparation entre les clusters. Plus la variance inter-cluster est grande par rapport à la variance intra-cluster, plus le score est élevé, ce qui indique que les clusters sont bien séparés.
- davies_bouldin_score : est calculé en mesurant la distance entre chaque paire de clusters et en comparant cette distance à la somme des rayons des deux clusters. Un score plus faible indique que les clusters sont plus compacts et séparés.

In [12]:
import warnings
warnings.filterwarnings('ignore')

On va commencer par le model K-Means

In [13]:
# On va definir l'experience du K-Means
mlflow.sklearn.autolog()

mlflow.set_experiment("K-Means_experiment")

with mlflow.start_run(run_name="default_K-Means_with_StandardScaler") as run:
    # On va creer un pipeline avec le model K-Means
    pipeline = create_pipeline(model = KMeans(), transform_for_num = StandardScaler())

    # On va entrainer le model
    pipeline.fit(data)

    # On va recuperer les predictions
    predictions = pipeline.predict(data)

    # On va ajouter les metrics
    mlflow.log_metric("silhouette_score", metrics.silhouette_score(data, predictions))
    mlflow.log_metric("inertia", pipeline['model'].inertia_)
    mlflow.log_metric("calinski_harabasz_score", metrics.calinski_harabasz_score(data, predictions))
    mlflow.log_metric("davies_bouldin_score", metrics.davies_bouldin_score(data, predictions))

mlflow.end_run()

2023/04/20 13:05:37 WARNING mlflow.utils: Truncated the value of the key `steps`. Truncated value: `[('preprocessor', ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 Index(['lib_etendu', 'bilan', 'montant_signe', 'signe', 'num_oper',
       'diff_ope_val'],
      dtype='obj...`
2023/04/20 13:05:37 WARNING mlflow.sklearn: Training metrics will not be recorded because training labels were not specified. To automatically record training metrics, provide training labels as inputs to the model training function.


In [ ]:
# On va definir l'experience du K-Means
mlflow.sklearn.autolog()

with mlflow.start_run(run_name="default_K-Means_with_MinMaxScaler") as run:
    # On va creer un pipeline avec le model K-Means
    pipeline = create_pipeline(model = KMeans(), transform_for_num = MinMaxScaler())

    # On va entrainer le model
    pipeline.fit(data)

    # On va recuperer les predictions
    predictions = pipeline.predict(data)

    # On va ajouter les metrics
    mlflow.log_metric("silhouette_score", metrics.silhouette_score(data, predictions))
    mlflow.log_metric("inertia", pipeline['model'].inertia_)
    mlflow.log_metric("calinski_harabasz_score", metrics.calinski_harabasz_score(data, predictions))
    mlflow.log_metric("davies_bouldin_score", metrics.davies_bouldin_score(data, predictions))

mlflow.end_run()

In [ ]:
with mlflow.start_run(run_name="default_K-Means_with_RobustScaler") as run:
    # On va creer un pipeline avec le model K-Means
    pipeline = create_pipeline(model = KMeans(), transform_for_num = RobustScaler())

    # On va entrainer le model
    pipeline.fit(data)

    # On va recuperer les predictions
    predictions = pipeline.predict(data)

    # On va ajouter les metrics
    mlflow.log_metric("silhouette_score", metrics.silhouette_score(data, predictions))
    mlflow.log_metric("inertia", pipeline['model'].inertia_)
    mlflow.log_metric("calinski_harabasz_score", metrics.calinski_harabasz_score(data, predictions))
    mlflow.log_metric("davies_bouldin_score", metrics.davies_bouldin_score(data, predictions))

mlflow.end_run()

In [ ]:
with mlflow.start_run(run_name="K-Means_model1_with_StandardScaler") as run:
    # On va creer un pipeline avec le model K-Means
    pipeline = create_pipeline(model = KMeans(), transform_for_num = StandardScaler())

    # On va entrainer le model
    pipeline.fit(data)

    # On va recuperer les predictions
    predictions = pipeline.predict(data)

    # On va ajouter les metrics
    mlflow.log_metric("silhouette_score", metrics.silhouette_score(data, predictions))
    mlflow.log_metric("inertia", pipeline['model'].inertia_)
    mlflow.log_metric("calinski_harabasz_score", metrics.calinski_harabasz_score(data, predictions))
    mlflow.log_metric("davies_bouldin_score", metrics.davies_bouldin_score(data, predictions))

mlflow.end_run()

# Voir les resultats sur le tableau de bord MLflow en utilisant nyngrok

In [20]:
# On va creer un tunnel pour acceder au tableau de bord MLflow
ngrok.kill()

NGROK_AUTH_TOKEN = "2OgxmHRxos2U37s8DO5FlG7Pn3D_3ih56ethLcFPMdEhTBxnS"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

ngrok_tunnel = ngrok.connect(addr='5000', proto='http', bind_tls=True)
print('MLflow Tracking UI: ', ngrok_tunnel.public_url)

t=2023-04-20T13:18:40+0000 lvl=warn msg="ngrok config file found at both XDG and legacy locations, using XDG location" xdg_path=C:\\Users\\ADMIN\\AppData\\Local/ngrok/ngrok.yml legacy_path=C:\\Users\\ADMIN\\.ngrok2\\ngrok.yml


MLflow Tracking UI:  https://83d1-197-253-215-109.ngrok-free.app


In [ ]:
!mlflow ui